# Advent of Code 2019, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202019%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [22]:
⍝ Helper functions and common settings
⎕IO←0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
line←{⊃lines ⍵}
sorted←{⍵[⍋⍵]}
sortbycol←⊢⌷⍨∘⊂∘⍒⌷⍤1
pairs←{↓(2÷⍨≢⍵) 2⍴⍵}
bin←{(32⍴2)⊤⍵}
dec←2∘⊥
and←{(bin ⍺)∧bin ⍵}
or←{(bin ⍺)∨bin ⍵}
rs←⊢∘-∘≢↑↓⍨∘-⍨
range←⊣+∘⍳-⍨

In [8]:
⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=min -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→──────┐
│Was OFF│
└───────┘

### Day 1: The Tyranny of the Rocket Equation
https://adventofcode.com/2019/day/1

In [7]:
DAY01←⍎¨⊃⎕NGET'data/2019/01.txt'1
Fuel←{0⌈¯2+⌊⍵÷3}

In [9]:
⊢part1←+/Fuel DAY01
assert 3328306=part1

3328306

In [10]:
⊢part2←+/{⍵=0:0⋄(Fuel ⍵)+∇Fuel ⍵}¨DAY01
assert 4989588=part2

4989588

### Day 2: 1202 Program Alarm
https://adventofcode.com/2019/day/2

The first of MANY intcode tasks.

In [15]:
DAY02←⍎line'data/2019/02.txt'

In [22]:
]dinput
IntcodeV1←{
    ⍺←0                                          ⍝ Intcode interpreter. 
    (op p1 p2 p3)←4↑⍺↓⍵                          ⍝ Skip anything before ip (⍺) and take 4 cells
    op∊1 2:(⍺+4)∇((op-1)⌷⍵[p1](+,×)⍵[p2])@p3 ⊢ ⍵ ⍝ Addition and multiplication
    op=99:⍵[0]                                   ⍝ Exit
}

In [23]:
DAY02[1 2] ← 12 2
⊢part1←IntcodeV1 DAY02
assert 10566835=part1

10566835

In [24]:
Day02p2←{19690720=IntcodeV1 ((⊃1↑⍵)@1 2)⊢⍺:⊃1↑⍵⋄⍺∇1↓⍵}
(noun verb)←DAY02 Day02p2 {,⍳⍵ ⍵} 100 
⊢part2←verb+100×noun
assert 2347=part2

2347

### Day 3: Crossed Wires
https://adventofcode.com/2019/day/3

In [12]:
DAY03←1(↑,∘⍎↓)¨⎕CSV'data/2019/03.txt' ⍝ Convert each string to a letter and a number.
OFFSETS←4 2⍴0 1 0 ¯1 1 0 ¯1 0

In [13]:
]dinput
Follow←{
    ⍺←0 0
    0=≢⍵:⍺
    (dir steps)←⊃⍵
    seq←,⌿OFFSETS['UDRL'⍳dir;]∘.×1+⍳steps
    origin←(≢seq)⍴⊂¯2↑∊⍺
    ⍺,∊(seq+origin)∇1↓⍵                  
}

In [14]:
(path1 path2)←pairs∘Follow¨↓DAY03

In [15]:
crossings←1↓∪path1∩path2 ⍝ Find intersections; drop the start point which is shared.

In [16]:
⊢part1←⌊/(+/|)¨crossings
assert 860=part1

860

In [17]:
⊢part2←⌊/+/(path1⍳crossings),⍪path2⍳crossings
assert 9238=part2

9238

### Day 4: Secure Container
https://adventofcode.com/2019/day/4

In [20]:
DAY04←236491 713787

In [23]:
⊢part1←+/{enc←(6⍴10)∘⊤⍵⋄(∧/2≤/enc)∧∨/2=/enc}¨(0⊃DAY04)range 1+1⊃DAY04
assert 1169=part1

1169

In [25]:
ngr←{2∊≢¨((1,2≠/⊢)⍵)⊂⍵} ⍝ contains pair not part of larger group

In [27]:
⊢part2←+/{enc←(6⍴10)∘⊤⍵⋄(∧/2≤/enc)∧ngr enc}¨(0⊃DAY04)range 1+1⊃DAY04
assert 757=part2

757

### Day 5: Sunny with a Chance of Asteroids
https://adventofcode.com/2019/day/5

Better get used to it ⍨.

In [30]:
DAY05←⊢⌿⍎¨⎕CSV'data/2019/05.txt'

In [35]:
]dinput
IntcodeV2←{ ⍝ Intcode interpreter, mk2. Call as: 0 IntcodeV2 code
    state←⍵
    ev←{⍺=0: state[⍵] ⋄ ⍵}              ⍝ Position or immediate mode
    op p1 p2 p3←4↑⍺↓⍵                   ⍝ Skip anything before and take 4 cells
    params←4 4 2 2 3 3 4 4 1            ⍝ Number of parameters by opcode
    ops←(1+⍳8),99                       ⍝ Valid opcodes
    m3 m2 m1 o2 o1←(5⍴10)⊤op            ⍝ Unpack the param modes
    op←10⊥o2 o1                         ⍝ Repack the opcode, to go from (say) 1001 to 1
    count←params[ops⍳op]                ⍝ Number of params
    parmod←m1 m2 m3,⍪p1 p2 p3           ⍝ Table combining modes and params
    d1 d2 d3←3↑ev/(¯1+count)↑parmod     ⍝ Pick relevant number of params, and apply modes
    ip←⍺+count                          ⍝ Advance ip by the width of current instr
    op∊1 2:ip∇((op-1)⌷d1(+,×)d2)@p3⊢⍵   ⍝ Addition and multiplicatin
    op=3:ip∇INPUT@p1⊢⍵                  ⍝ Input
    op=4:d1,ip∇⍵                        ⍝ Output
    op∊5 6:⍵∇⍨ip d2⌷⍨(op-5)⌷d1(≠,=)0    ⍝ Jumps 
    op∊7 8:ip∇((op-7)⌷d1(<,=)d2)@p3⊢⍵   ⍝ Comparison < or =
    op=99:⍬                             ⍝ Exit
}

In [36]:
INPUT←1
⊢part1←⊃¯1↑0 IntcodeV2 DAY05
assert 16209841=part1

16209841

In [39]:
INPUT← 5
⊢part2←⊃0 IntcodeV2 DAY05
assert 8834787=part2

8834787

### Day 6: Universal Orbit Map
https://adventofcode.com/2019/day/6

In [53]:
DAY06←↓⍉↑'\w+'⎕S'&'¨lines'data/2019/06.txt'
PARENTS←⊃⍳/⊖DAY06
Path←{3::⍵⋄⍵,∇⍵⊃PARENTS}

In [55]:
⊢part1←≢∊Path¨PARENTS
assert part1=292387

292387

In [56]:
⊢part2←{¯2+≢⍺(∪~∩)⍵}/Path¨(1⊃DAY06)⍳'YOU' 'SAN'
assert 433=part2

433